In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipaddress
from datetime import datetime
import time
import keras
import re
from sklearn.preprocessing import OneHotEncoder

In [19]:
data = pd.read_csv("./data_full.csv", sep = ";", error_bad_lines=False)

b'Skipping line 10780784: expected 10 fields, saw 11\nSkipping line 10793876: expected 10 fields, saw 11\n'
b'Skipping line 10821923: expected 10 fields, saw 11\n'
C:\Users\ASUS\Anaconda3\envs\tensorflow-gpu\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
catalog = pd.read_csv("./catalog.csv", sep = ";", error_bad_lines=False)

In [21]:
data.head()

,timestamp,ip,track_id,cookie,live,user_id,referer,uagent,rightholder,author_id
0,1517778000,230.59.74.120,10912114,42536dc7a8578b0cfac05f704977429a,0,NaN,https%3A%2F%2Frutube.ru%2Fvideo%2F0f5c9c5839b1...,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2,1480930
1,1517778000,218.222.225.36,10885813,8f92f3f2b1e7a2498761b8cb3b1d03c3,0,NaN,https%3A%2F%2Frutube.ru%2Fvideo%2Fd8068436dcf5...,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,2,301323
2,1517778000,55.63.241.224,6948236,336d5ebc5436534e61d16e63ddfca327,-,2.231332e+09,http%3A%2F%2Fandroid.rutube.ru%2F,okhttp/2.6.0,-,-
3,1517778000,230.34.98.99,8481402,cf8dd809edf104873a1c57921b34f25e,0,NaN,https%3A%2F%2Frutube.ru%2Fvideo%2F1058cf249e46...,Mozilla/5.0 (Linux; Android 7.1.2; Redmi 4X Bu...,NaN,721346
4,1517778000,232.3.41.226,10908060,4ca525a1d60de6d74e3c32529fe47ab5,0,NaN,https%3A%2F%2Fyastatic.net%2Fvideo-player%2F0x...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,135,599848


In [22]:
catalog.head()

,idec,track_id,video_id,series_id,title,season,episode,episode_global,date_efir,duration
0,PR592505,10258734,cd59084eb50e6ebe9fe20ad04e0c17db,1.045685e+10,"\tБыть или не быть, 1 сезон, 1 серия",1.0,1.0,1.0,02.04.2017 22:00:34,NaN
1,PR609630,10843255,a5d2e6bae4752d9ec0fae9b53f00700d,1.059247e+10,NaN,11.0,1.0,1.0,NaN,NaN
2,PR597128,10573153,bcd45606c1f7cbb5cb9d1dbd7d346cc3,1.047956e+10,NaN,7.0,1.0,1.0,06.09.2017 22:46:05,NaN
3,PR257331,6617650,609217ac7b8c4e55d44c4a29bf52554b,NaN,"""Интерны. История болезни""",NaN,NaN,NaN,12.06.2012 22:00:44,NaN
4,PR529698,7991493,bde3620ba038b3e8dd41d03bf516a7eb,1.024102e+10,#ЖАННАПОЖЕНИ,1.0,1.0,101.0,05.09.2015 13:30:00,NaN


In [23]:
def add_device(df):
    regexpDesktop = re.compile('(Windows|Linux|Mozilla)')
    regexpMobile = re.compile('(iPhone|RutubeAndroid|okhttp|CFNetwork|UCBrowser)')
    regexpTablet = re.compile('(iPad)')
    
    df['device'] = pd.Series('0', index=df.index)
    
    for i, row in df.iterrows():
        try:
            if regexpDesktop.search(df['uagent'].iloc[i]):
                df['device'][i] = 'desktop'
            elif regexpMobile.search(df['uagent'].iloc[i]):
                df['device'][i] = 'mobile'
            elif regexpTablet.search(df['uagent'].iloc[i]):
                df['device'][i] = 'mobile'
        except:
            pass
            
    return df

In [24]:
def mergeData(data1, data2, field):
    data1[field] = pd.to_numeric(data1[field], errors = 'coerce')
    data2[field] = pd.to_numeric(data2[field], errors = 'coerce')
    return pd.merge(data1, data2, left_on=field, right_on=field)

In [25]:
def convIpToNumber(data):
    return data.apply(lambda x: int(ipaddress.IPv4Address(x)))

In [26]:
def applyDate(x):
    try:
        data = time.strptime(x, "%d.%m.%Y %H:%M:%S")
        return int(time.mktime(data))
    except:
        return 1e-6

In [27]:
def confDateStringToTimestamp(data):
    return data.apply(lambda x: applyDate(x))

In [28]:
def filter_having_more_than_n_views(data, user_column = "ip", seq_len = 4):
    by_user = data.groupby(user_column)
    by_user = by_user.filter(lambda x: len(x) >= seq_len)
    return by_user

def extract_sequences_by_user(data, user_column = "ip"):
    return data.groupby(user_column)

In [108]:
def vocab_size(catalog, field):
    return catalog[field].unique().shape[0]
vocab_size = vocab_size(catalog, "track_id") * 2
print(vocab_size)

55644


In [77]:
def one_hot_encoder(data, y_field, vocab_size):
    onehot = OneHotEncoder()
    expanded = np.expand_dims(data[y_field][data[y_field].notnull()].values, axis = 1)
    print(len(expanded))
    print(expanded)
    onehot.fit(expanded)
    return onehot

In [30]:
data['ip'] = convIpToNumber(data['ip'])
prepared = filter_having_more_than_n_views(data)

C:\Users\ASUS\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [79]:
prepared["track_id"] = pd.to_numeric(prepared["track_id"], errors = 'coerce', downcast = 'integer')

C:\Users\ASUS\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
def prepare_data(data, catalog):
    
    data = mergeData(data, catalog, "track_id")
    
    data['ip'] = convIpToNumber(data['ip'])
    
    data = filter_having_more_than_n_views(data)
    
    data = data.drop(['live', 'user_id', 'rightholder'], axis=1)
    
    data['date_efir'] = confDateStringToTimestamp(data['date_efir'])
    
    data = add_device(data)
    
    return data

print(prepare_data(data.head(10000), catalog))

NameError: name 'mergeData' is not defined

In [119]:
def create_x_video_vectors(x):
     return np.expand_dims(x["track_id"].values, axis=0).reshape(1, len(x))

    #     return np.repeat(np.expand_dims(x["track_id"].values, axis=0), len(x), axis = 0).reshape(len(x), len(x))

print(create_x_video_vectors(data.head(3)))

def create_x(x):
    return create_x_video_vectors(x)

[['10912114' '10885813' '6948236']]


In [142]:
one_hot = one_hot_encoder(prepared, "track_id", vocab_size)

def create_y(y):
    y = one_hot.transform(np.expand_dims(y["track_id"][y["track_id"].notnull()].values, axis = 1)).toarray()
    return y

7967447
[[10912114.]
 [10885813.]
 [ 6948236.]
 ...
 [10917309.]
 [ 7634638.]
 [10915645.]]


In [143]:
def train_generator(seq_by_user, x_len = 3):
    for ip, group in seq_by_user:
        group = group.sort_values("timestamp")
        x = create_x(group[0:x_len])
        y = create_y(group[x_len:])
        yield (x, y)
        
generation = next(train_generator(extract_sequences_by_user(prepared)))
print(generation[0].shape)
print(generation[1].shape)

(1, 3)
(1, 248868)


In [146]:
vocab_size = 248868
print(vocab_size)

emb_input = keras.layers.Input(shape=(3,))

embedding = keras.layers.Embedding(vocab_size, 100, input_length=3)(emb_input)

avg = keras.layers.AveragePooling1D(pool_size=3)(embedding)

#input_num_cat = keras.layers.Input(shape = num_cat_shape)
#concat = keras.layers.Concat(embedding, input_num_cat)
# gru_1 = keras.layers.GRU(128, return_sequences=True)(embedding)
# gru_2 = keras.layers.GRU(128)(gru_1)
dense_1 = keras.layers.Dense(1024, activation = "tanh")(avg)
dense_2 = keras.layers.Dense(1024, activation = "tanh")(dense_1)
dense_3 = keras.layers.Dense(1024, activation = "tanh")(dense_2)
drop = keras.layers.Dropout(0.5)(dense_3)
dense_4 = keras.layers.Dense(vocab_size, activation = "softmax")(drop)

model = keras.models.Model(inputs=emb_input, outputs=dense_4)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

model.summary()

model.fit_generator(train_generator(extract_sequences_by_user(prepared)), steps_per_epoch=len(prepared), epochs=1, verbose=1)



248868
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 3)                 0         
_________________________________________________________________
embedding_20 (Embedding)     (None, 3, 100)            24886800  
_________________________________________________________________
average_pooling1d_4 (Average (None, 1, 100)            0         
_________________________________________________________________
dense_53 (Dense)             (None, 1, 1024)           103424    
_________________________________________________________________
dense_54 (Dense)             (None, 1, 1024)           1049600   
_________________________________________________________________
dense_55 (Dense)             (None, 1, 1024)           1049600   
_________________________________________________________________
dropout_14 (Dropout)         (None, 1, 1024)           0         
___

ValueError: Error when checking target: expected dense_56 to have 3 dimensions, but got array with shape (1, 248868)